# 1. 사전 준비

In [1]:
!pip install kiwipiepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/bert

/content/drive/MyDrive/bert


# 2. Data Load

In [4]:
import pandas as pd
import json
import re

In [90]:
with open("validation.json", "r", encoding="UTF-8") as f:
    data = json.load(f)

In [91]:
p = re.compile("#(.*)#")

In [92]:
res = []
for frame in data:
    for sentence in frame["sentences"]:
        m = p.search(sentence["origin_text"])
        if m == None:
            res.append([sentence["origin_text"], sentence["types"]])
        else:
            start = m.span()[0]
            end = m.span()[1]
            temp = sentence["origin_text"][0:start] + "OO" + sentence["origin_text"][end:]
            res.append([temp, sentence["types"]])

In [93]:
immoral = []
sexual = []
abusement = []

for r in res:
  if "IMMORAL_NONE" in r[1]:
    immoral.append([r[0], 0])
  elif "SEXUAL" in r[1]:
    sexual.append([r[0], 2])
  elif "ABUSE" in r[1] or "VIOLENCE" in r[1]:
    abusement.append([r[0], 1])


In [94]:
len(immoral), len(sexual), len(abusement)

(20344, 2306, 3713)

In [95]:
immoral[:5], abusement[:5], sexual[:5], 

([['명절이면 좀 일찍 마쳐주지', 0],
  ['요즘은 왜 미세먼지가 별로 없지?', 0],
  ['호주랑 중국이랑 싸우고 있어서 중국에서 호주산석탄을 수입 안해서 그렇대', 0],
  ['하긴 그럴 것 같긴 해', 0],
  ['와 어제 그 댄서 퍼포먼스 장난 아니더라', 0]],
 [['웃을 때마다 역겨워서 주먹 날리고 싶음', 1],
  ['처묵고 싶나 밴태야', 1],
  ['처맞고싶나 이따위 저질행동하지마라 주지도않으끼다', 1],
  ['넌 정말 최악이야', 1],
  ['좋은 짱깨들은 이미 천안문에서 죽었음 리얼루', 1]],
 [['그래서 니가 육봉을 다른 가시나들한테 뿌리고 다니는구나', 2],
  ['발정난 암캐 찾아요~', 2],
  ['절라 개꼴이네 니', 2],
  ['스벌 딴넘한테는 잘준다는 소문이있던데', 2],
  ['니 다리벌려라고 소문낫다 알고나 다니라', 2]])

# 3. 형태소 분리

In [96]:
from kiwipiepy import Kiwi
kiwi = Kiwi()

In [97]:
for i in range(len(immoral)):
  result = kiwi.tokenize(immoral[i][0])
  temp_token = []
  for x in result:
    temp_token.append(x.form)
  immoral[i].append(temp_token)

In [98]:
for i in range(len(abusement)):
  result = kiwi.tokenize(abusement[i][0])
  temp_token = []
  for x in result:
    temp_token.append(x.form)
  abusement[i].append(temp_token)

In [99]:
for i in range(len(sexual)):
  result = kiwi.tokenize(sexual[i][0])
  temp_token = []
  for x in result:
    temp_token.append(x.form)
  sexual[i].append(temp_token)

In [100]:
immoral[:3], abusement[:3], sexual[:3]

([['명절이면 좀 일찍 마쳐주지', 0, ['명절', '이', '면', '좀', '일찍', '마치', '어', '주', '지']],
  ['요즘은 왜 미세먼지가 별로 없지?',
   0,
   ['요즘', '은', '왜', '미세', '먼지', '가', '별로', '없', '지', '?']],
  ['호주랑 중국이랑 싸우고 있어서 중국에서 호주산석탄을 수입 안해서 그렇대',
   0,
   ['호주',
    '랑',
    '중국',
    '이랑',
    '싸우',
    '고',
    '있',
    '어서',
    '중국',
    '에서',
    '호주',
    '산',
    '석탄',
    '을',
    '수입',
    '안',
    '하',
    '어서',
    '그렇',
    '대']]],
 [['웃을 때마다 역겨워서 주먹 날리고 싶음',
   1,
   ['웃', '을', '때', '마다', '역겹', '어서', '주먹', '날리', '고', '싶', '음']],
  ['처묵고 싶나 밴태야', 1, ['처', '묵', '고', '싶', '나', '밴태야']],
  ['처맞고싶나 이따위 저질행동하지마라 주지도않으끼다',
   1,
   ['처',
    '맞',
    '고',
    '싶',
    '나',
    '이',
    '따위',
    '저질',
    '행동',
    '하',
    '지',
    '말',
    '어라',
    '주',
    '지',
    '도',
    '않',
    '으',
    '끼',
    '다']]],
 [['그래서 니가 육봉을 다른 가시나들한테 뿌리고 다니는구나',
   2,
   ['그래서',
    '니',
    '가',
    '육',
    '봉',
    '을',
    '다른',
    '가시나',
    '들',
    '한테',
    '뿌리',
    '고',
    '다니',
    '는',
    '구나']],
  ['발정난 암캐 찾아요~',

# 4. csv 저장

In [101]:
df_immoral = pd.DataFrame(immoral)
df_abusement = pd.DataFrame(abusement)
df_sexual = pd.DataFrame(sexual)

In [102]:
df_immoral.head()

,0,1,2
0,명절이면 좀 일찍 마쳐주지,0,"[명절, 이, 면, 좀, 일찍, 마치, 어, 주, 지]"
1,요즘은 왜 미세먼지가 별로 없지?,0,"[요즘, 은, 왜, 미세, 먼지, 가, 별로, 없, 지, ?]"
2,호주랑 중국이랑 싸우고 있어서 중국에서 호주산석탄을 수입 안해서 그렇대,0,"[호주, 랑, 중국, 이랑, 싸우, 고, 있, 어서, 중국, 에서, 호주, 산, 석..."
3,하긴 그럴 것 같긴 해,0,"[하긴, 그렇, 을, 것, 같, 기, ᆫ, 하, 어]"
4,와 어제 그 댄서 퍼포먼스 장난 아니더라,0,"[와, 어제, 그, 댄서, 퍼포먼스, 장난, 아니, 더라]"


In [103]:
df_abusement.head()

,0,1,2
0,웃을 때마다 역겨워서 주먹 날리고 싶음,1,"[웃, 을, 때, 마다, 역겹, 어서, 주먹, 날리, 고, 싶, 음]"
1,처묵고 싶나 밴태야,1,"[처, 묵, 고, 싶, 나, 밴태야]"
2,처맞고싶나 이따위 저질행동하지마라 주지도않으끼다,1,"[처, 맞, 고, 싶, 나, 이, 따위, 저질, 행동, 하, 지, 말, 어라, 주,..."
3,넌 정말 최악이야,1,"[너, ᆫ, 정말, 최악, 이, 야]"
4,좋은 짱깨들은 이미 천안문에서 죽었음 리얼루,1,"[좋, 은, 짱깨, 들, 은, 이미, 천안문, 에서, 죽, 었, 음, 리얼, 루]"


In [104]:
df_sexual.head()

,0,1,2
0,그래서 니가 육봉을 다른 가시나들한테 뿌리고 다니는구나,2,"[그래서, 니, 가, 육, 봉, 을, 다른, 가시나, 들, 한테, 뿌리, 고, 다니..."
1,발정난 암캐 찾아요~,2,"[발정, 나, ᆫ, 암캐, 찾, 어요, ~]"
2,절라 개꼴이네 니,2,"[절라, 개꼴이네, 니]"
3,스벌 딴넘한테는 잘준다는 소문이있던데,2,"[스벌, 딴넘, 한테, 는, 잘, 주, ᆫ다는, 소문, 이, 있, 던데]"
4,니 다리벌려라고 소문낫다 알고나 다니라,2,"[니, 다리, 벌리, 어, 이, 라고, 소문나, 엇, 다, 알, 고, 나, 어, 다..."


In [105]:
df_immoral.to_csv("validation_immoral.csv", encoding="utf-8-sig")
df_abusement.to_csv("validation_abusement.csv", encoding="utf-8-sig")
df_sexual.to_csv("validation_sexual.csv", encoding="utf-8-sig")